In [40]:
import os
import numpy as np
import rasterio as rio
from copy import deepcopy

In [41]:
dataset = r"/home/abetatos/Escritorio/TFM_Dataset"

outp = "4.DatasetPost"
inp = "3.Basedataset"

inpath = os.path.join(dataset, inp)
outpath = os.path.join(dataset, outp)

In [42]:
FEATURES =  os.listdir(inpath)
LOCATIONS = os.listdir(os.path.join(inpath, "HS"))

In [43]:
def get_array(inpath_file):
    if not os.path.isfile(inpath_file): 
        return np.array([])
    with rio.open(inpath_file, "r") as src: 
        array = src.read()
    return array

def get_arrays(feature):
    arrays = []
    for location in LOCATIONS:
        inpath_file = os.path.join(inpath, feature, location, feature + ".tif")
        arrays.append(get_array(inpath_file))
    return arrays

def get_lims(feature):
    arrays = get_arrays(feature)
    maximum = np.array([a.max() for a in arrays if a.any()]).max()
    for a in arrays:
        a[a<=-9999.] = np.inf
    minimum = np.array([a.min() for a in arrays if a.any()]).min()
    return maximum, minimum

In [44]:
def rescale(array, minimum, maximum): 
    mask = array==-9999.
    array = (array - minimum)/(maximum - minimum)
    array[mask] = -9999.
    return array

In [45]:
for feature in FEATURES:
    maximum, minimum = get_lims(feature)
    print(feature, maximum, minimum)
    for location in LOCATIONS:
        inpath_file = os.path.join(inpath, feature, location, feature + ".tif")
        outpath_file = os.path.join(outpath, feature, location, feature + ".tif")

        if not os.path.isfile(inpath_file): 
            continue
        
        with rio.open(inpath_file, "r") as src:
            array = src.read()[0]
            profile = src.profile
        
        os.makedirs(os.path.dirname(outpath_file), exist_ok=True)
        with rio.open(outpath_file, "w", **profile) as src: 

            if feature != "HS":
                if feature == "TPIWGW": 
                    array[array>5] = 5.
                    array[(array > -9999) & (array < -5)] = -5
                src.write(rescale(array, minimum, maximum), 1)
            else:
                array[(array > -9999) & (array < 0)] = 0
                src.write(array, 1)

DEM 3408.71 959.74744
Voronoi 3.105957 0.02
SCE 0.9726027 0.0
FFSnow 180.0 0.0
DEMSNOW 6.17763 -0.1650578
TPI 1.0 -1.0
Slope 33.00405 0.0
TPIWGW 8201.83 -9337.114
ProbStat 0.9975658 0.06996767
AspectSin 1.0 -1.0
Aspect 359.99948 0.0
HS 26.533447 -18.737549
AspectCos 1.0 -1.0
